In [ ]:
# for numerical work
import pandas as pd
import numpy as np

import pymongo

import datetime
import json

from pandas.io.json import json_normalize
from pymongo import MongoClient

import pickle

# load the database credentials from file
with open('../creds/creds.json') as json_data:
    creds = json.load(json_data)
    
client = MongoClient(creds['connection_string'])
    
# get the new credit card and interac requests
all_requests = list(client['ml']['requestEvents60'].find())

# flatten into one object per event
flat_requests = [{'request': rs['request'], 
                  'event': event} for rs in all_requests for event in rs['events']]

# flatten requests into a dataframe
all_events = pd.DataFrame(json_normalize(flat_requests))

# create a dataframe with the results
df_with_id = all_events

# sort by request id and date
df_with_id = df_with_id.sort_values(by=['request._id','event.created'])

# calculate the previous event time and the time between events
df_with_id['previous_event_time'] = df_with_id.groupby(['request._id'])['event.created'].shift(1)
df_with_id['event.time_since_last_event'] = pd.to_numeric(df_with_id['event.created']-df_with_id['previous_event_time'])*1e-9

# replace string versions of infinity with proper inf object
df_with_id = df_with_id.replace('Infinity', np.inf)

# convert columns that should be to numeric
df_with_id['request.metadata.amount'] = pd.to_numeric(df_with_id['request.metadata.amount'])
df_with_id['request.metadata.rate'] = pd.to_numeric(df_with_id['request.metadata.rate'])
df_with_id['request.metadata.cents'] = pd.to_numeric(df_with_id['request.metadata.cents'])
df_with_id['request.value'] = pd.to_numeric(df_with_id['request.value'])
df_with_id['event.metadata.amount'] = pd.to_numeric(df_with_id['event.metadata.amount'])
df_with_id['event.metadata.rate'] = pd.to_numeric(df_with_id['event.metadata.rate'])
df_with_id['event.metadata.cents'] = pd.to_numeric(df_with_id['event.metadata.cents'])
df_with_id['event.value'] = pd.to_numeric(df_with_id['event.value'])

# get the days since november
df_with_id['event.days_since_nov'] = df_with_id['event.created'].apply(lambda x: (x - datetime.datetime(year=2017,month=11,day=1)).days)

# replace older bitcoin labels with new format
df_with_id.loc[df_with_id['event.eventLabel'].str.lower() == 'bitcoin', 'event.eventLabel'] = 'BTC'

# create unique category identifiers
df_with_id['event.category_action_label'] = df_with_id['event.eventCategory']+'_'+df_with_id['event.eventAction']+'_'+df_with_id['event.eventLabel']
df_with_id['event.category_action'] = df_with_id['event.eventCategory']+'_'+df_with_id['event.eventAction']

# drop columns that contain list/array values because they can't be processed
list_drops = [col for col in df_with_id.columns if df_with_id[col].apply(lambda x: type(x)).value_counts().index[0] == "<class 'list'>"]
df_with_id = df_with_id.drop(list_drops, axis=1)

# drop some other columns
df_with_id = df_with_id.drop(['event.metadata.authResponseEIN.body.data.token_type','event.metadata.authResponseEIN.headers.map.content-type'], axis=1)

# categorical columns that need to be converted to binary
categorical_columns = ['event.category_action',
                        'event.category_action_label',
                        'event.metadata.addressCity',
                        'event.metadata.addressCountry',
                        'event.metadata.addressProvince',
                        'event.metadata.city',
                        'event.metadata.country',
                        'event.metadata.currency',
                        'event.metadata.instrument',
                        'event.metadata.mongoResponse.product',
                        'event.metadata.product',
                        'event.metadata.productId',
                        'event.metadata.prossessorError.billingDetails.city',
                        'event.metadata.prossessorError.billingDetails.country',
                        'event.metadata.prossessorError.billingDetails.state',
                        'event.metadata.prossessorError.card.type',
                        'event.metadata.prossessorError.currencyCode',
                        'event.metadata.prossessorResponse.billingDetails.city',
                        'event.metadata.prossessorResponse.billingDetails.country',
                        'event.metadata.prossessorResponse.billingDetails.province',
                        'event.metadata.prossessorResponse.billingDetails.state',
                        'event.metadata.prossessorResponse.card.cardType',
                        'event.metadata.prossessorResponse.card.type',
                        'event.metadata.prossessorResponse.card_type',
                        'event.metadata.prossessorResponse.currency',
                        'event.metadata.prossessorResponse.currencyCode',
                        'event.metadata.province',
                        'event.metadata.requestParams.currency',
                        'event.metadata.requestParams.product',
                        'event.metadata.type']


unique_columns = ['event.metadata.bankName',
                 'event.metadata.cardHolder',
                 'event.metadata.cardId',
                 'event.metadata.cardName',
                 'event.metadata.cardNumberLastFour',
                 'event.metadata.cardPrefix',
                 'event.metadata.cardSuffix',
                 'event.metadata.email',
                 'event.metadata.firstName',
                 'event.metadata.fullName',
                 'event.metadata.lastName',
                 'event.metadata.mongoResponse.email',
                 'event.metadata.name',
                 'event.metadata.prossessorError.card.cardExpiry.month',
                 'event.metadata.prossessorError.card.cardExpiry.year',
                 'event.metadata.prossessorError.card.lastDigits',
                 'event.metadata.prossessorError.card.type',
                 'event.metadata.prossessorResponse.card.cardExpiry.month',
                 'event.metadata.prossessorResponse.card.cardExpiry.year',
                 'event.metadata.prossessorResponse.card.cardType',
                 'event.metadata.prossessorResponse.card.lastDigits',
                 'event.metadata.prossessorResponse.card.type',
                 'event.metadata.prossessorResponse.card_expiry_month',
                 'event.metadata.prossessorResponse.card_expiry_year',
                 'event.metadata.prossessorResponse.card_suffix',
                 'event.metadata.prossessorResponse.card_type',
                 'event.metadata.prossessorResponse.profile.email',
                 'event.metadata.prossessorResponse.profile.firstName',
                 'event.metadata.prossessorResponse.profile.lastName',
                 'event.metadata.requestParams.card_id',
                 'event.metadata.requestParams.email']

numerical_per_currency = ['event.metadata.amount',
                         'event.metadata.blockioResponse.data.amount_sent',
                         'event.metadata.blockioResponse.data.amount_withdrawn',
                         'event.metadata.lastTradedPx',
                         'event.metadata.mongoResponse.amount',
                         'event.metadata.mongoResponse.price',
                         'event.metadata.price',
                         'event.metadata.prossessorResponse.amount',
                         'event.metadata.rate',
                         'event.metadata.requestParams.amount',
                         'event.metadata.requestParams.price',
                         'event.metadata.requestParams.product_amount']

numerical_overall = ['event.metadata.cents',
                     'event.metadata.prossessorResponse.charge_amount',
                     'event.metadata.requestParams.charge_amount',
                     'event.value',
                     'event.time_since_last_event',
                     'event.days_since_nov']

all_columns = list(set(categorical_columns + numerical_per_currency + numerical_overall + unique_columns))

In [ ]:
# convert columns to either numeric or categorical
def convert_to_numeric_or_lower_str(column):
    
        try:
            return pd.to_numeric(col)

        except:
            return col.str.lower()

df_with_id = df_with_id.apply(convert_to_numeric_or_lower_str, axis=1)

In [ ]:
# summarize the columns where unique values matter
unique_data = df_with_id[['request._id']+unique_columns]

def n_unique(series):
    
    return series.dropna().unique().size

def n_NaN(series):
    
    return np.sum(series.isnull())

unique_summary = unique_data.groupby(['request._id'])[unique_columns].agg([n_unique, n_NaN])
unique_summary.columns = [col[0] if col[1] == '' else col[0]+'_'+col[1] for col in unique_summary.columns.ravel()]

In [ ]:
# summarize numerical by currency
numerical_by_currency_data = df_with_id[['request._id','event.category_action_label']+numerical_per_currency]
numerical_by_currency_data.dropna(axis=0, how='all', subset=numerical_per_currency, inplace=True)
groupby_agg = numerical_by_currency_data.groupby(['request._id','event.category_action_label'], as_index=False)[numerical_per_currency].agg(['mean','median','max','min','std'])
groupby_agg.columns = [col[0] if col[1] == '' else col[0]+'_'+col[1] for col in groupby_agg.columns.ravel()]
groupby_agg = groupby_agg.reset_index()
groupby_agg = groupby_agg.melt(id_vars=['request._id','event.category_action_label'])
groupby_agg['variable'] = groupby_agg['event.category_action_label']+'_'+groupby_agg['variable']
groupby_agg.drop('event.category_action_label', axis=1, inplace=True)
groupby_agg = groupby_agg.pivot(index='request._id', columns='variable', values='value').reset_index().set_index('request._id')

In [ ]:
df = df_with_id[all_columns]

df = convert_to_numeric_or_lower_str(df)

same_cols = generate_same_columns_df(df)

filled_df = fill_same_values(df, same_cols)

filled_df.head()

In [ ]:
[col for col in sorted(df.columns)]

In [ ]:
first_level_cols = [col for col in same_cols.col.unique() if col.count('.') < 3]
drop_cols = []
levels = np.max([col.count('.') for col in same_cols.col.unique()])

for i in range(levels):
    for col in [col for col in same_cols.col.unique() if col.count('.') == i]:

        drop_cols += list(same_cols[(same_cols.col == col) & (same_cols.same_percent_of_other > 0.95)].other_col.unique())

drop_cols = list(set(drop_cols))

sorted(clean_df.columns.drop(drop_cols))

In [ ]:
import re

def convert(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower().replace('.','_')

def combine_same_name_columns(df):
    
    clean_df = df
    
    columns = df.columns
    
    drops = []
    
    for col in columns:
        
        if col not in drops:
            other_cols = columns.drop(col)
            for other_col in other_cols:
                if other_col not in drops:
                    if convert(col) == convert(other_col):
                        print(col,'is same as',other_col, 'filling values and dropping', other_col)
                        clean_df.loc[clean_df[col].isnull(), col] = clean_df[clean_df.isnull()][other_col]
                        clean_df = clean_df.drop(other_col, axis=1)
                        drops.append(other_col)
                
    
    return clean_df

In [ ]:
cleaner_df = combine_same_name_columns(clean_df)

In [ ]:
# calculate the columns that are the same as other columns
def same_column_mappings(df, min_similarity=0.95):

    results = []
    
    processed = []
    
    # convert columns to numeric if possible
    def try_numeric(col):
        try:
            return pd.to_numeric(col)
        except:
            return col
    
    df = df.apply(try_numeric, axis=1)
    
    df = df.replace('Infinity', np.nan)

    # get the columns sorted by number of actual values present
    columns = pd.DataFrame(df.count()).reset_index().sort_values(by=0, ascending=False)['index']
    
    # do the same for each type of column
    numerical = df.select_dtypes(include=['float64','int64'])
    non_numerical = df.select_dtypes(include='object')
    
    numerical_cols = pd.DataFrame(numerical.count()).reset_index().sort_values(by=0, ascending=False)['index']
    non_numerical_cols = pd.DataFrame(non_numerical.count()).reset_index().sort_values(by=0, ascending=False)['index']

    # for each column 
    for col in columns:
        
        # if it hasn't been processed yet
        if col not in processed:
            result = {'column': col, 'same_cols': []}
            processed.append(col)
            print("Processing: ", col)

            if df[col].dtype == 'float64' or df[col].dtype == 'int64':

                for other_col in numerical_cols[numerical_cols != col].values:
                    if other_col not in processed:
                        sub_df = df[[col,other_col]].dropna(how='any')

                        try:
                            same = np.sum(pd.to_numeric(sub_df[col]) == pd.to_numeric(sub_df[other_col]))
                            same_percentage = same/sub_df[other_col].dropna().size

                            if same_percentage >= min_similarity:

                                result['same_cols'].append(other_col)
                                processed.append(other_col)

                        except Exception as e:
                            print(e)

            else:

                for other_col in non_numerical_cols[non_numerical_cols != col].values:
                    if other_col not in processed:
                        sub_df = df[[col,other_col]].dropna(how='any')

                        try:
                            same = np.sum(sub_df[col] == sub_df[other_col])
                            same_percentage = same/sub_df[other_col].dropna().size

                            if same_percentage >= min_similarity:

                                result['same_cols'].append(other_col)
                                processed.append(other_col)

                        except Exception as e:
                            print(e)

            if len(result['same_cols']) > 0: 
                results.append(result)

    return results

same_mappings = same_column_mappings(df_with_id[all_columns])

In [ ]:
same_mappings

In [ ]:
def combine_same_cols(df, same_mappings):
    
    for col_set in same_mappings:
        
        column = col_set['column']
        other_cols = col_set['same_cols']
        
        for other_col in other_cols:
            
            df.loc[df[column].isnull(), column] = df[df[column].isnull()][other_col].values
            df = df.drop(other_col, axis=1)
            print("Dropped", other_col)
            
    return df


clean_df = combine_same_cols(df_with_id[all_columns], same_mappings)

[col for col in sorted(clean_df.columns)]

In [ ]:
# look at places where the email is different from the processor email
ev = df[df['event.metadata.email'] != df['event.metadata.prossessorResponse.profile.email']][['event.category_action_label','event.metadata.email','event.metadata.prossessorResponse.profile.email']].dropna()
ev = ev.drop('event.category_action_label', axis=1)

ev.drop_duplicates()

In [ ]:
# sort ascending by request_id and descending by time
subset = subset.sort_values(by=['request_id','created'], ascending=[True, False])

# grab the emails
user_emails = subset['request_email']

# get the fraudulent emails
def get_fraud_labels(user_emails):
    '''Remove whitelisted or test emails'''

    # get all the events related to the requests aka within 60 minutes before the first request for the users who mader requests
    bl_emails = [r['email'] for r in list(client['production']['emailBlacklistCollection'].find({'level': 'BLOCKED'}))]
 
    return np.array([1 if user in bl_emails else 0 for user in user_emails])

# get the fraud labels
fraud = get_fraud_labels(user_emails)

# drop the time and email columns
subset = subset.drop(['request_email','created'], axis = 1)
subset = subset.reset_index(drop=True)

# hacky way to number the sequence events prior to each interac request with 0 being the request and 10 being the 10th event prior to the request
subset['index_int'] = subset.index
event_index = subset.groupby('request_id')['index_int'].agg(lambda x: list(np.abs(min(x)-x)))
all_index = [[item] if type(item) == type(int) else list(item) for item in event_index]
all_index = [item for sublist in all_index for item in sublist]
subset['timesteps'] = all_index

print("Dataframe is",np.sum(subset.memory_usage())*1e-9,'gigabytes in memory')

print("Saving to hdf5 file")

subset.to_hdf('../lstm_data_prep_pipeline/results/all_request_events.hdf5', 'table')

print("Done")